In [ ]:
# import osmnx
import osmnx as ox
import geopandas as gpd
import rasterio
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# define the place name
place_name:str = "Graz, Austria"
# get the boundary polygon of Graz
graz_gdf = ox.geocode_to_gdf(place_name)
graz_gdf.to_file("graz_boundary.geojson", driver="GeoJSON")
# change the crs to an Austrian metric CRS EPSG: 31256
graz_gdf = graz_gdf.to_crs(epsg=31256)






'\nThe author\'s of the paper obtained the road and drainage network from the Ministry of Agriculture\nand the Addis Ababa city land administration. We use the logical equivalent from OSM.\nPaper speaks of "drainage" also in respect to recreation. For that, we will choose rivers and streams.\ntags: waterway=river, waterway=stream, waterway=canal for drainage network\n\nFor the road network, accessibility and pollution is relevant, so will choose the drivable road network.\nnetwork_type=\'drive\' for drivable road network (excluding footways, cycleways, etc.)\n\n\n\n'

In [ ]:
"""
The author's of the paper obtained the road and drainage network from the Ministry of Agriculture
and the Addis Ababa city land administration. We use the logical equivalent from OSM.
Paper speaks of "drainage" also in respect to recreation. For that, we will choose rivers and streams.
tags: waterway=river, waterway=stream, waterway=canal for drainage network

For the road network, accessibility and pollution is relevant, so will choose the drivable road network.
network_type='drive' for drivable road network (excluding footways, cycleways, etc.)
"""

# get the road network within Graz
road_graph = ox.graph_from_place(place_name, network_type='drive')
road_gdf = ox.graph_to_gdfs(road_graph, nodes=False, edges=True)
road_gdf = road_gdf.to_crs(epsg=31256)
road_gdf.to_file("graz_road_network.geojson", driver="GeoJSON")

# get the drainage network within Graz
#https://wiki.openstreetmap.org/wiki/Key:waterway
drainage_tags = {'waterway': ['river', 'stream', 'canal']}
drainage_gdf = ox.features_from_place(place_name, tags=drainage_tags)
drainage_gdf = drainage_gdf.to_crs(epsg=31256)
drainage_gdf.to_file("graz_drainage_network.geojson", driver="GeoJSON")

# we only want lines, no polygons or points
drainage_gdf = drainage_gdf[drainage_gdf.geometry.type.isin(['LineString', 'MultiLineString'])]
drainage_gdf = drainage_gdf[['geometry', 'waterway', 'name']] # keep names
drainage_gdf.to_file("graz_drainage_network.geojson", driver="GeoJSON")

# print summary
print(f"Graz boundary: {graz_gdf.shape[0]} features")
print(f"Graz road network: {road_gdf.shape[0]} features")
print(f"Graz drainage network: {drainage_gdf.shape[0]} features")

Graz boundary: 1 features
Graz road network: 11255 features
Graz drainage network: 997 features


In [ ]:
"""
Make raster base layer for Graz with 10m resolution
"""
# define raster parameters
raster_res = 10  # 10 meters
minx, miny, maxx, maxy = graz_gdf.total_bounds

  Using cached cligj-0.7.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached click_plugins-1.1.1.2-py2.py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/25.4 MB ? eta -:--:--
   ----- ---------------------------------- 3.4/25.4 MB 16.7 MB/s eta 0:00:02
   --------- ------------------------------ 6.0/25.4 MB 14.2 MB/s eta 0:00:02
   -------------- ------------------------- 9.2/25.4 MB 15.0 MB/s eta 0:00:02
   ------------------- -------------------- 12.3/25.4 MB 14.8 MB/s eta 0:00:01
   ------------------------ --------------- 15.5/25.4 MB 14.7 MB/s eta 0:00:01
   ------------------------------ --------- 19.4/25.4 MB 15.1 MB/s eta 0:00:01
   ----------------------------------- ---- 22.8/25.4 MB 15.5 MB/s eta 0:00:01
   ---------------------------------------  25.4/25.4 MB 15.6 MB/s eta 0:00:01
   ---------------------------------------- 25.4/25.4 MB 14.9 MB/s eta 0:00:00
Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
   ------------------------------


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
